In [3]:
import gurobipy as gp
import pandas as pd
from code_map import data_handling
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os
import plotly.graph_objects as go
import plotly.express as px


In [ ]:
# get access to data from the yr-weather module which is used to get weather data from the yr.no API. I have installed the module from the terminal using the command: pip install yr-weather
import yr_weather.yr_weather as yr_weather
# Replace with your own User-Agent. See MET API Terms of Service for correct User-Agents.
headers = {
    "User-Agent": "sander_meland_master_thesis/1.0 sander.meland@student.uib.no"
}

my_client = yr_weather.Locationforecast(headers=headers)

# Get air temperature in Oslo, Norway
oslo_temp = my_client.get_air_temperature(59.91, 10.75)

print(oslo_temp)
# Example output: 8.0

# Get full forecast for Oslo, Norway
forecast = my_client.get_forecast(59.91, 10.75)

# Select the forecast for the time right now (as it's possible to select a time further in the future)
forecast_now = forecast.now()

# You can now select from multiple data points. As an example, we show air pressure and wind speed.
pressure = forecast_now.details.air_pressure_at_sea_level
wind_speed = forecast_now.details.wind_speed

print(f"Air pressure at sea level in Oslo, Norway, is {pressure} hPa and the wind speed is {wind_speed} m/s")

### The code under is from when i tried to download a csv file and preprocess it. I dont think it will ever be useful again, but i dont want to delete it just in case.

In [9]:
weather_data = pd.read_csv('../master-data/weather_data/weather_5_meters.csv', sep=';')

In [10]:
weather_data.columns

Index(['Navn', 'Stasjon', 'Tid(norsk normaltid)', 'Nedbør (1 t)',
       'Lufttemperatur'],
      dtype='object')

In [35]:
def preprocess_weather_data(df : pd.DataFrame):
    #df.drop(df.loc[df["Navn"] ==  'Data er gyldig per 06.12.2023 (CC BY 4.0), Meteorologisk institutt (MET)'], inplace=True)
    df = df.copy()
    df.rename(columns={'Tid(norsk normaltid)' : 'Time (Local)', \
                       "Navn" : "station_name", \
                       "Stasjon" : "station", \
                       "Lufttemperatur" : "air_temp", \
                       "Vindretning" : "wind_direction" , \
                       "Nedbør (1 t)" : "precipitation"}, inplace=True)
    # convert from str to float
    for column_name in ['precipitation',  'air_temp']:
        df[column_name] = df[column_name].str.replace(',', '.')
        df[column_name] = df[column_name].astype(float)

    df["area"] = df["station_name"].apply(lambda x : "NO1" if x == 'Gardermoen' else "NO2" if x == 'Kristiansand - Sømskleiva' else "NO3" if x == 'Trondheim - Risvollan' else "NO4" if x == 'Tromsø' else "NO5" if x == 'Bergen - Florida' else "None")
    # get this format : 01.01.2022 01:00
    format = '%d.%m.%Y %H:%M'
    df['Time (Local)'] = pd.to_datetime(df['Time (Local)'], format= format, utc=True)
    df['Time (Local)'] = df['Time (Local)'].dt.tz_convert('Europe/Oslo')
    df.sort_values(by=['Time (Local)'], inplace=True)
    df = df.loc[(df["Time (Local)"] >= pd.Timestamp(2023, 6, 1, 0, 0, 0, tz='Europe/Oslo')) & (df["Time (Local)"] < pd.Timestamp(2023, 7, 1, 0, 0, 0, tz='Europe/Oslo')) ]
    df.reset_index(drop=True, inplace= True)

    df.drop(['station_name', 'station'], axis=1, inplace=True)
    return df

### It seems like the data misses several hours - will probably be better to try API



In [36]:
weather_data_df = preprocess_weather_data(weather_data)

In [25]:
display(weather_data_df)

,station_name,station,Time (Local),precipitation,air_temp
0,Trondheim - Risvollan,SN68230,2023-06-01 00:00:00+02:00,"0,2",4
1,Kristiansand - Sømskleiva,SN39150,2023-06-01 00:00:00+02:00,0,"15,3"
2,Tromsø,SN90450,2023-06-01 00:00:00+02:00,0,"0,9"
3,Bergen - Florida,SN50540,2023-06-01 00:00:00+02:00,0,"9,4"
4,Gardermoen,SN4780,2023-06-01 00:00:00+02:00,0,"12,3"
...,...,...,...,...,...
3595,Tromsø,SN90450,2023-06-30 23:00:00+02:00,0,"17,9"
3596,Bergen - Florida,SN50540,2023-06-30 23:00:00+02:00,0,"13,9"
3597,Trondheim - Risvollan,SN68230,2023-06-30 23:00:00+02:00,0,"11,6"
3598,Kristiansand - Sømskleiva,SN39150,2023-06-30 23:00:00+02:00,0,"15,9"


In [26]:
weather_data_df["station_name"].unique()

array(['Trondheim - Risvollan', 'Kristiansand - Sømskleiva', 'Tromsø',
       'Bergen - Florida', 'Gardermoen'], dtype=object)

In [37]:
type(weather_data_df["precipitation"].iloc[0])

numpy.float64

In [38]:
weather_data_df.head(10)

,Time (Local),precipitation,air_temp,area
0,2023-06-01 00:00:00+02:00,0.2,4.0,NO3
1,2023-06-01 00:00:00+02:00,0.0,15.3,NO2
2,2023-06-01 00:00:00+02:00,0.0,0.9,NO4
3,2023-06-01 00:00:00+02:00,0.0,9.4,NO5
4,2023-06-01 00:00:00+02:00,0.0,12.3,NO1
5,2023-06-01 01:00:00+02:00,0.0,3.6,NO3
6,2023-06-01 01:00:00+02:00,0.0,11.5,NO1
7,2023-06-01 01:00:00+02:00,0.0,13.5,NO2
8,2023-06-01 01:00:00+02:00,0.0,9.2,NO5
9,2023-06-01 01:00:00+02:00,0.1,0.4,NO4
